In [1]:
%load_ext autoreload
%autoreload 2
from data.dataloader import get_tokenizer
from model.model import GPT
from data.utils import Trie
import pickle
import numpy as np 
import torch

from data.dataloader import SearchData
from torch.utils.data import DataLoader
from model.env import Env

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = get_tokenizer()
actor = GPT(tokenizer=tokenizer, vocab_size=len(tokenizer), block_size=100).cuda()
reader = GPT(tokenizer=tokenizer, vocab_size=len(tokenizer), block_size=100).cuda()
reader_pt = torch.load('checkpoints/baseline/gt_0.001/baseline_23.pt')
reader.load_state_dict(reader_pt)
optimizer = torch.optim.Adam(actor.parameters(), lr=1e-3)


In [3]:
with open('data/trie.pkl', 'rb') as f:
    trie = pickle.load(f)
with open('data/dataset.pkl', 'rb') as f:
    dataset = pickle.load(f)
    _, database = dataset

In [4]:
dataset = SearchData('data/train.pkl', tokenizer)
loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0)

In [5]:
env = Env(reader, database, trie, tokenizer, max_len=100).cuda()

In [8]:
from train_rl import train_policy_gradient
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/debugv2')

In [9]:
train_policy_gradient(actor, optimizer, loader, env, writer=writer, reward_type='ce', gamma=0.9)

KeyboardInterrupt: 

In [12]:
from train_rl import train_policy_gradient_iterative
actor = GPT(tokenizer=tokenizer, vocab_size=len(tokenizer), block_size=100).cuda()
reader = GPT(tokenizer=tokenizer, vocab_size=len(tokenizer), block_size=100).cuda()
optimizer = torch.optim.Adam(actor.parameters(), lr=1e-3)
env = Env(reader, database, trie, tokenizer, max_len=100).cuda()
env_opt = torch.optim.Adam(reader.parameters(), lr=1e-3)

writer = SummaryWriter('runs/pg_acc_iterative')
train_policy_gradient_iterative(actor, optimizer, env_opt, loader, env, writer=writer, reward_type='acc', gamma=0.9)

Episode 878: reward=-2.2637, policy_loss=0.0012, accuracy=0.0625:  35%|███▌      | 878/2500 [04:30<08:01,  3.37it/s] 